In [1]:
import torch
import math
import numpy as np
import pickle
import random

torch.manual_seed(11)
np.random.seed(11)
random.seed(11)

### Data Generation

In [2]:
DATA_NUM = 16000
CLUSTER_NUM = 4
EXPERT_NUM = 8
PATCH_NUM = 4
PATCH_LEN = 50

In [3]:
features = torch.zeros(CLUSTER_NUM, PATCH_LEN)
x = np.random.randn(PATCH_LEN) 
x /= np.linalg.norm(x)
current_x = []
for i in range(CLUSTER_NUM):
    features[i] = torch.tensor(x)
    current_x.append(x)
    x = np.random.randn(PATCH_LEN) 
    x /= np.linalg.norm(x)
    for x_prev in current_x:
        x -= x.dot(x_prev) * x_prev
    x /= np.linalg.norm(x)

centers = torch.zeros(CLUSTER_NUM, PATCH_LEN)
for i in range(CLUSTER_NUM):
    centers[i] = torch.tensor(x)
    if i!=3:
        current_x.append(x)
        x = np.random.randn(PATCH_LEN) 
        x /= np.linalg.norm(x)
        for x_prev in current_x:
            x -= x.dot(x_prev) * x_prev
        x /= np.linalg.norm(x)


data = []
labels = []
train_cluster_idx = [[] for x in range(CLUSTER_NUM)]
test_cluster_idx = [[] for x in range(CLUSTER_NUM)]

for i in range(DATA_NUM*2):
    y = np.random.choice([-1,1], 1)[0] 
    k = np.random.choice(list(range(0,CLUSTER_NUM)))
    
    if i < DATA_NUM:
        train_cluster_idx[k].append(i)
    else:
        test_cluster_idx[k].append(i-DATA_NUM)

    # Noise patch
    xi = torch.tensor(np.random.normal(0, 2/math.sqrt(PATCH_LEN), size=(PATCH_LEN)))
    
    # Feature noise patch
    pos_or_neg = np.random.choice([-1,1], 1)[0]
    k_noise = np.random.choice(list(set(range(0,CLUSTER_NUM))-set([int(k)])))
    alpha, beta, gamma = np.random.uniform(0.5,2), np.random.uniform(1,2), np.random.uniform(0.5,2)

    x = torch.stack([features[k]*y*alpha, centers[k]*beta, xi, 
                     pos_or_neg*features[k_noise]*gamma])
    
    # random permutation
    idx = torch.randperm(len(x))
    x = x[idx].flatten()

    data.append(x)
    labels.append(y)

data = torch.stack(data)
print(data.shape)

labels = torch.tensor(labels)
labels[labels==-1] = 0
print(labels.shape)

training_data = data[:DATA_NUM,:].unsqueeze(1).float()
test_data = data[DATA_NUM::].unsqueeze(1).float()
print(training_data.shape, test_data.shape)

training_labels = labels[:DATA_NUM]
test_labels = labels[DATA_NUM:]
print(training_labels.shape, test_labels.shape)

training_data, test_data = training_data.cuda(), test_data.cuda()
training_labels, test_labels = training_labels.cuda(), test_labels.cuda()
training_labels = training_labels.long()
test_labels = test_labels.long()
centers, features = centers.cuda(), features.cuda()
training_data *= 10
test_data *= 10
# Below does not affect training, just for plotting
centers *= 10
features *= 10

torch.Size([32000, 200])
torch.Size([32000])
torch.Size([16000, 1, 200]) torch.Size([16000, 1, 200])
torch.Size([16000]) torch.Size([16000])


In [4]:
torch.save(training_data, 'synthetic_data_s4/train_data.pt')
torch.save(training_labels, 'synthetic_data_s4/train_labels.pt')

torch.save(test_data, 'synthetic_data_s4/test_data.pt')
torch.save(test_labels, 'synthetic_data_s4/test_labels.pt')

torch.save(centers, 'synthetic_data_s4/centers.pt')
torch.save(features, 'synthetic_data_s4/features.pt')

with open("synthetic_data_s4/train_cluster", "wb") as fp:  
    pickle.dump(train_cluster_idx,fp)
    
with open("synthetic_data_s4/test_cluster", "wb") as fp:  
    pickle.dump(test_cluster_idx,fp)